## Initial EnvironmentSetup

In [1]:
# 1. Create a python environment

# !conda create -y --name bedrock-agents-mac python=3.11.8
# !conda init && activate bedrock-agents-mac
# !conda install -n bedrock-agents-mac ipykernel --update-deps --force-reinstall -y
# !conda install -c conda-forge ipython-sql

## OR
# !python -m venv venv
# !source venv/bin/activate  # On Windows, use `venv\Scripts\activate`

In [2]:
# 2. Install dependencies

# !pip install -r requirements.txt

In [1]:
# 3. Import necessary libraries and load environment variables
from dotenv import load_dotenv, find_dotenv, set_key
import os
import sagemaker
import boto3
import json
import os
import pandas as pd

from utils.bedrock import BedrockLLMWrapper

# loading environment variables that are stored in local file
local_env_filename = 'dev.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['S3_BUCKET_NAME'] = os.getenv('S3_BUCKET_NAME')
os.environ['AWS_ACCOUNT'] = os.getenv('AWS_ACCOUNT')

REGION = os.environ['REGION']
S3_BUCKET_NAME = os.environ['S3_BUCKET_NAME']
AWS_ACCOUNT = os.environ['AWS_ACCOUNT']

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


In [2]:
import botocore.config
config = botocore.config.Config(
    connect_timeout=600,  # 10 minutes
    read_timeout=600,     # 10 minutes
    retries={'max_attempts': 3}
)

session = boto3.Session(region_name=REGION)

# Create clients/session
sagemaker_session = sagemaker.Session(boto_session=session)
bedrock_agent_client = session.client('bedrock-agent', config=config)
bedrock_agent_runtime_client = session.client('bedrock-agent-runtime', config=config)
bedrock_runtime_client = session.client('bedrock-runtime', config=config)
bedrock_client = session.client('bedrock', config=config)
lambda_client = session.client('lambda', config=config)
iam_resource = session.resource('iam')
iam_client = session.client('iam')
athena_client = session.client('athena')
s3_client = session.client('s3')


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


In [15]:
import sagemaker

# Get the default bucket
session = sagemaker.Session()
default_bucket = session.default_bucket()
print(f"Default SageMaker bucket: {default_bucket}")

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Default SageMaker bucket: amazon-sagemaker-165361166149-us-west-2-7c123d407319


## Create S3 bucket

In [6]:
def create_s3_bucket_if_not_exists(s3_client, bucket_name, region):
    try:
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"Bucket {bucket_name} already exists")
    except s3_client.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            try:
                # For regions other than us-east-1, we need to specify LocationConstraint
                if region != 'us-east-1':
                    s3_client.create_bucket(
                        Bucket=bucket_name,
                        CreateBucketConfiguration={'LocationConstraint': region}
                    )
                else:
                    s3_client.create_bucket(Bucket=bucket_name)
                print(f"Created bucket {bucket_name}")
            except Exception as create_error:
                print(f"Error creating bucket: {create_error}")
                raise
        else:
            print(f"Error checking bucket: {e}")
            raise

# Initialize S3 client and create bucket

# create_s3_bucket_if_not_exists(s3_client, S3_BUCKET_NAME, REGION)


In [5]:
# create the following folders in the bucket
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='metadata/use_cases/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='metadata/er_diagram/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='metadata/sql_table_definition/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='uploads/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='models/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='results/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='athena_results/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='raw/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='ml_datasets/')
s3_client.put_object(Bucket=S3_BUCKET_NAME, Key='mlflow/')

ClientError: An error occurred (AccessDenied) when calling the PutObject operation: User: arn:aws:sts::165361166149:assumed-role/datazone_usr_role_cxmdg4c04jc8sw_b42dcr7rrxtxcg/SageMaker is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::amazon-sagemaker-165361166149-us-west-2-7c123d407319/metadata/use_cases/" because no identity-based policy allows the s3:PutObject action

## Upload all test-files to S3

In [8]:
# upload all files from data/use_case_data to s3 uploads folder
for file in os.listdir('../data/use_case_data'):
    s3_client.upload_file(os.path.join('../data/use_case_data', file), 
                          S3_BUCKET_NAME, 
                          f'uploads/{file}')


## Upload use case details to S3

In [9]:
# upload file data/use_case_details.jsonl to s3 metadata/use_cases folder

file_path = '../data/use_case_details.jsonl'

# S3 bucket name
bucket_name = S3_BUCKET_NAME
key_prefix = 'metadata/use_cases/use_case_details.jsonl'

#upload file to s3
s3_client.upload_file(file_path, 
                      bucket_name, 
                      key_prefix)

## Create ECR repositories for Docker images

In [ ]:
# create repository for Docker images
# Initialize ECR client
ecr_client = session.client('ecr')

# List of repositories to create
repositories = [
    'automatedinsights/lambda_businessanalyst',
    'automatedinsights/lambda_dataengineer',
    'automatedinsights/lambda_datascientist',
    'automatedinsights/lambda_supervisor'
]

# Function to create ECR repository if it doesn't exist
def create_ecr_repository_if_not_exists(ecr_client, repository_name):
    try:
        # Try to describe the repository to see if it exists
        ecr_client.describe_repositories(repositoryNames=[repository_name])
        print(f"Repository {repository_name} already exists")
    except ecr_client.exceptions.RepositoryNotFoundException:
        try:
            # Create the repository if it doesn't exist
            response = ecr_client.create_repository(
                repositoryName=repository_name,
                imageScanningConfiguration={'scanOnPush': True},
                encryptionConfiguration={'encryptionType': 'AES256'}
            )
            print(f"Created repository {repository_name}")
            return response
        except Exception as e:
            print(f"Error creating repository {repository_name}: {e}")
            raise
    except Exception as e:
        print(f"Error checking repository {repository_name}: {e}")
        raise

# Create repositories
for repo in repositories:
    create_ecr_repository_if_not_exists(ecr_client, repo)

## Create Athena database if it doesn't exist

In [ ]:
def create_glue_database_if_not_exists(glue_client, database_name):
    try:
        # Check if database exists
        glue_client.get_database(Name=database_name)
        print(f"Database {database_name} already exists")
    except glue_client.exceptions.EntityNotFoundException:
        try:
            # Create database if it doesn't exist
            glue_client.create_database(
                DatabaseInput={
                    'Name': database_name,
                    'Description': 'Database for Automated Insights project',
                    'LocationUri': f's3://{database_name}/athena_results/'
                }
            )
            print(f"Created database {database_name}")
        except Exception as e:
            print(f"Error creating database: {e}")
            raise
    except Exception as e:
        print(f"Error checking database: {e}")
        raise

# Initialize Glue client
glue_client = session.client('glue')

# Create database (using S3 bucket name as database name for consistency)
database_name = S3_BUCKET_NAME.replace('-', '_')  # Glue databases can't have hyphens
create_glue_database_if_not_exists(glue_client, database_name)

## Setup Bedrock Application Inference Profiles

Wwe will create 1 inference profile for each agent, and 1 inference profile for each agent evaluation, and 1 inference profile for the multi-agent evaluation. Therefore in total we will have 7 inference profiles.

In [ ]:
# list inference profiles
# bedrock_client.list_inference_profiles(typeEquals='APPLICATION')

# delete all inference profiles that start with 'ClaudeSonnet'
response = bedrock_client.list_inference_profiles(typeEquals='APPLICATION')
for profile in response.get('inferenceProfileSummaries', []):
    if profile.get('inferenceProfileName', '').startswith('ClaudeSonnet'):
        bedrock_client.delete_inference_profile(
            inferenceProfileIdentifier=profile['inferenceProfileArn']
        )

# List remaining profiles
bedrock_client.list_inference_profiles(typeEquals='APPLICATION')

In [ ]:
# 1. create multi-agent evaluation application inference profile
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-3-5-sonnet-20241022-v2:0"
cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetAutomatedInsights',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
    ]
)

mac_eval_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'MAC_EVAL_PROFILE_ARN', mac_eval_profile_arn)

In [ ]:
# Test the inference profile
from utils.bedrock import BedrockLLMWrapper
bedrock_llm = BedrockLLMWrapper(model_id=mac_eval_profile_arn, 
                                max_token_count=2000,
                                temperature=0,
                                region=REGION,
                                session=session
                            )
bedrock_llm.generate("Hello, how are you?")

In [ ]:
# 2. create Data Engineer application inference profile with cross regional routing

cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetDataEngineer',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
        {
            'key': 'agent',
            'value': 'DataEngineer'
        },
    ]
)
cr_app_inf_profile_response['inferenceProfileArn']
dataengineer_agent_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'DATAENGINEER_AGENT_PROFILE_ARN', dataengineer_agent_profile_arn)

In [ ]:
# 3. create Data Engineer Eval application inference profile with cross regional routing

cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetDataEngineerEval',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
        {
            'key': 'agent',
            'value': 'DataEngineer'
        },
        {
            'key': 'evaluation',
            'value': 'DataEngineerEval'
        },
    ]
)
cr_app_inf_profile_response['inferenceProfileArn']
dataengineer_agent_eval_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'DATAENGINEER_AGENT_EVAL_PROFILE_ARN', dataengineer_agent_eval_profile_arn)

In [ ]:
# 4. create Business Analyst application inference profile with cross regional routing

cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetBusinessAnalyst',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
        {
            'key': 'agent',
            'value': 'BusinessAnalyst'
        },
    ]
)
cr_app_inf_profile_response['inferenceProfileArn']
businessanalyst_agent_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'BUSINESSANALYST_AGENT_PROFILE_ARN', businessanalyst_agent_profile_arn)

In [ ]:
# 5. create Business Analyst Eval application inference profile with cross regional routing

cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetBusinessAnalystEval',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
        {
            'key': 'agent',
            'value': 'BusinessAnalyst'
        },
        {
            'key': 'evaluation',
            'value': 'BusinessAnalystEval'
        },
    ]
)
cr_app_inf_profile_response['inferenceProfileArn']
businessanalyst_agent_eval_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'BUSINESSANALYST_AGENT_EVAL_PROFILE_ARN', businessanalyst_agent_eval_profile_arn)


In [ ]:
# 6. create Data Scientist application inference profile with cross regional routing

cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetDataScientist',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
        {
            'key': 'agent',
            'value': 'DataScientist'
        },
    ]
)
cr_app_inf_profile_response['inferenceProfileArn']
datascientist_agent_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'DATASCIENTIST_AGENT_PROFILE_ARN', datascientist_agent_profile_arn)


In [ ]:
# 7. create Data Scientist Eval application inference profile with cross regional routing

cr_inf_profile_arn = f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}'
cr_app_inf_profile_response = bedrock_client.create_inference_profile(
    inferenceProfileName='ClaudeSonnetDataScientistEval',
    description='Application profile for Claude Sonnet 3.5',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectName',
            'value': 'huthmac-AutomatedInsights'
        },
        {
            'key': 'agent',
            'value': 'DataScientist'
        },
        {
            'key': 'evaluation',
            'value': 'DataScientistEval'
        },
    ]
)
cr_app_inf_profile_response['inferenceProfileArn']
datascientist_agent_eval_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
# save to dev.env
set_key(local_env_filename, 'DATASCIENTIST_AGENT_EVAL_PROFILE_ARN', datascientist_agent_eval_profile_arn)
